In [1]:
import pandas as pd
import altair as alt

#CO2 emission yearly change (metric tons)
climate_data = pd.read_csv('absolute-change-co2.csv')
climate_data = climate_data[climate_data['Year'] >= 1900]
countries = ['Asia', 'China', 'Africa', 'Sweden', 'Australia', 'South America', 'Angola', 'India', 'New Zealand', 'United States', 'Europe', 'North America']
country_dfs = []
for i in range(len(countries)):
    country_i = climate_data.loc[climate_data['Entity'] == countries[i]]
    country_dfs.append(country_i)
climate_data = pd.concat(country_dfs)
climate_data.reset_index(inplace = True)
climate_data.drop(columns = ['index', 'Code'], inplace = True)
climate_data = climate_data.dropna()


In [2]:
#Yearly CO2 emissions per capita (metric tons)
climate_per_capita = pd.read_csv('co-emissions-per-capita.csv')
climate_per_capita = climate_per_capita[climate_per_capita['Year'] >= 1900]
countries = ['Asia', 'China', 'Africa', 'Sweden', 'Australia', 'South America', 'Angola', 'India', 'New Zealand', 'United States', 'Europe', 'North America']
country_dfs = []
for i in range(len(countries)):
    country_i = climate_per_capita.loc[climate_per_capita['Entity'] == countries[i]]
    country_dfs.append(country_i)
climate_per_capita = pd.concat(country_dfs)
climate_per_capita.reset_index(inplace = True)
climate_per_capita.drop(columns = ['index', 'Code'], inplace = True)

#Combining two dfs into one and converting it into a csv 
per_cap_series = pd.Series(climate_per_capita['Per capita CO2 emissions'], name='Per capita CO2 emissions')
total_data = pd.concat([climate_data, per_cap_series], axis=1)
oxford_climate_data = total_data.to_csv(path_or_buf = '/Users/samuelembry/Downloads/oxford_climate_data')

In [5]:
#Reading in csv created above, might not need to create separate csv actually
climate_data = pd.read_csv('oxford_climate_data')
climate_data.drop(columns = 'Unnamed: 0', inplace = True)

In [8]:
#Global average temperature anomaly by year compared to 1961-1990 average global temp
global_temp = pd.read_csv('temperature-anomaly.csv')
global_temp = global_temp[global_temp['Year'] >= 1900]
global_temp.reset_index(inplace = True)
global_temp.drop(columns = ['index', 'Code'], inplace = True)
global_temp.head()

,Entity,Year,Median temperature anomaly from 1961-1990 average,Upper bound (95% CI),Lower bound (95% CI)
0,Global,1900,-0.203,-0.147,-0.272
1,Global,1901,-0.259,-0.208,-0.326
2,Global,1902,-0.402,-0.355,-0.461
3,Global,1903,-0.479,-0.431,-0.543
4,Global,1904,-0.520,-0.471,-0.580
